Get original DataFrame

In [ ]:
import sys
import plotly.graph_objects as go
import pandas as pd

import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import median_absolute_error
sys.path.insert(0, os.path.abspath('../..'))

from python_scripts.api_calls import fetch_item_to_df, fetch_items, get_cookie_from_blob


dailyCookie = get_cookie_from_blob()
items = fetch_items()

current_item = fetch_item_to_df(items[4], dailyCookie)
#print(items[4])

#print(current_item.tail())
#print(non_aggregated_item.tail())

df = current_item

os. getcwd()

convert an array of values into a dataset matrix

In [ ]:

def create_dataset(dataset, look_back=1):
 dataX, dataY = [], []
 #print(len(dataset))
 for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
 return np.array(dataX), np.array(dataY)


#printx,printy = create_dataset(dataset, 1)
#print(printx)

#Takes in np array and returns new orignal array with shifted values (look_back)

Loads and normalises dataset

In [ ]:
processed_df = df["price_usd"]
tf.random.set_seed(0)



dataset = processed_df.values
dataset = dataset.astype('float32')
print(dataset)
#print(processed_df)
#normalize the dataset
dataset = dataset.reshape(-1, 1)
print(dataset)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
print(dataset)

split into training and testing

In [ ]:

train_size = int(len(dataset) * 0.7)
test_size = int(len(dataset) * 0.9)

train, test = dataset[0:train_size], dataset[train_size:test_size]

print(len(train), len(test))



# train_size = int(len(dataset) * 0.67)

# X_train, X_test = X[:split_point], X[split_point:]
# y_train, y_test = y[:split_point], y[split_point:]

Reshapes into X=t and Y=t+1

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
print((testX))
print((testY))
counter = 0
for i in range(len(train)-look_back-1):
        counter+=1
        


Reshapes input for samples, timesteps and features

In [ ]:
#print(trainX.shape[1])

#trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
#testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
#print(trainX)


Creates and fit the LSTM network

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=2, batch_size=1, verbose=2)


Makes predictions, inverts predictions, and calculate root mean sqaured error

In [ ]:
#trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions
#trainPredict = scaler.inverse_transform(trainPredict)
unscaledtrainY = scaler.inverse_transform([trainY])
unscaledtestPredict = scaler.inverse_transform(testPredict)
unscaledtestY = scaler.inverse_transform([testY])
rowunscaledtestY = unscaledtestY.reshape(-1,1)
# calculate root mean squared error
#trainScore = np.sqrt(mean_squared_error(scaledtrainY[0], trainPredict[:,0]))

#print(unscaledtestY, unscaledtestPredict)

#print(testPredict)
#print(testY)
print(unscaledtestY)

Metric calculation

In [ ]:
testScore = np.sqrt(mean_squared_error(unscaledtestY[0], unscaledtestPredict[:,0]))  #Root mean squared error
r2 = r2_score(unscaledtestY[0], unscaledtestPredict[:, 0]) #R2 score - This provides an indication of the goodness of fit and therefore a measure of how well unseen samples are likely to be predicted by the model. It is the proportion of the variance in the dependent variable that is predictable from the independent variables.
explained_variance = explained_variance_score(unscaledtestY[0], unscaledtestPredict[:, 0])  #Explained variance score  this measures the proportion to which a mathematical model accounts for the variation (dispersion) of a given data set. It is the proportion of the variance in the dependent variable that is predictable from the independent variables
medae = median_absolute_error(unscaledtestY[0], unscaledtestPredict[:, 0])
mae = mean_absolute_error(unscaledtestY[0], unscaledtestPredict[:, 0])
mape = np.mean(np.abs((unscaledtestY[0] - unscaledtestPredict[:, 0]) / unscaledtestY[0])) * 100

print('Test Score: %.2f RMSE' % (testScore))
print('R2 Score: %.2f' % (r2))
print('Explained Variance: %.2f' % (explained_variance))
print('Median Absolute Error: %.2f' % (medae))
print('Mean Absolute Error: %.2f' % (mae))
print('Mean Absolute Percentage Error: %.2f' % (mape)+'%')

Shifts predicitons and plots graphs

In [ ]:
# # shift train predictions for plotting
# trainPredictPlot = np.empty_like(dataset)

# trainPredictPlot[:, :] = np.nan

# #trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

plt.grid()
plt.title("LSTM - Predicted vs Actual")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.plot(unscaledtestPredict , label = "Predicted") #ȳ
plt.plot(rowunscaledtestY, label = "Actual") #ground truth values
plt.legend(loc="upper left")
plt.show()
print(testScore)